In [40]:
import pandas as pd
import sqlalchemy
import faker

In [41]:
import mysql.connector

In [42]:
from mysql.connector import Error

In [43]:
import json

In [44]:
import sqlite3

In [45]:
# Primero convertiremos nuestra informacion en el archivo csv, en una dataframe para el uso en jupyter

df = pd.read_csv("2014_VehículosMatriculadosBDD.csv", encoding='latin1')
df.head()

,PROVINCIA,SERVICIO,CLASE,COMBUSTIBLE,MARCA,MODELO,ASIENTOS,TONELAJE,ESTRATONE,ESTRASIENTOS
0,SUCUMBIOS,ALQUILER,Trailer,Diesel,INTERNATIONAL,2006,0,45.0,15 1/4 Y MÁS,0 A 10
1,GUAYAS,ALQUILER,Trailer,Diesel,INTERNATIONAL,2011,0,40.0,15 1/4 Y MÁS,0 A 10
2,CARCHI,ALQUILER,Camión,Diesel,HINO,2003 y anteriores,0,12.0,10 1/4 A 15,0 A 10
3,COTOPAXI,PARTICULAR,Camión,Diesel,HINO,2003 y anteriores,0,6.5,6 1/4 A 10,0 A 10
4,EL ORO,PARTICULAR,Camión,Diesel,CHEVROLET,2003 y anteriores,0,6.0,3 1/4 A 6,0 A 10


In [46]:
#comprobamos las dimenciones:

df.shape

(1752712, 10)

In [47]:
#eliminamos los duplicados:

df=df.drop_duplicates()

In [48]:
# REVISAMOS SI NO TENEMOS DATOS NULOS

df.isnull().sum()

PROVINCIA       0
SERVICIO        0
CLASE           0
COMBUSTIBLE     0
MARCA           0
MODELO          0
ASIENTOS        0
TONELAJE        0
ESTRATONE       0
ESTRASIENTOS    0
dtype: int64

In [49]:
# OBSERVAMOS EL TIPO DE DATOS CON LOS QUE SE TRABAJARA

df.dtypes

PROVINCIA        object
SERVICIO         object
CLASE            object
COMBUSTIBLE      object
MARCA            object
MODELO           object
ASIENTOS          int64
TONELAJE        float64
ESTRATONE        object
ESTRASIENTOS     object
dtype: object

In [50]:
# comprovamos las dimenciones finales

df.shape

(125534, 10)

In [ ]:
''EXPORTAMOS LA DATAFRAME DE JUPYTER A MONGODB''

In [51]:
# primero procedemos con la coneccion a mongodb

import pandas as pd
from pymongo import MongoClient

mi_registro = "vehiculos6"

client = MongoClient("mongodb://localhost:27017/")
db=client["midb"]
collection=db["mi_registro"]

In [52]:
# se procede a exportar la dataframe de jupyter a mongodb

data = df.to_dict(orient="records")
collection = db[mi_registro]
collection.insert_many(data)

client.close()

In [ ]:
''PROCEDEMOS A IMPORTAR LOS DATOS DE MONGODB HACIA JUPYTER''

In [14]:
import pymongo
import pandas as pd

In [15]:
mi_registro = "vehiculos2"

client = MongoClient("mongodb://localhost:27017/")
db = client["midb"]
collection = db["mi_registro"]

cursor = collection.find()
lista_documentos = list(cursor)
df = pd.DataFrame(lista_documentos)

print(df.head())

Empty DataFrame
Columns: []
Index: []


In [ ]:
''EXPORTAMOS DE MONGODB A UN JSON''

In [53]:
import pymongo
import json

mi_registro = "vehiculos6"

cliente = pymongo.MongoClient("mongodb://localhost:27017/")
db = cliente["midb"]
coleccion = db["vehiculos2"]

datos = list(coleccion.find())

for documento in datos:
    if "_id" in documento:
        del documento["_id"]

with open("vehiculos_registro.json", "w") as f:
    json.dump(datos, f, indent=4, default=str)

print("Datos exportados a datos_vehiculos.json")

Datos exportados a datos_vehiculos.json


In [ ]:
'' CONVERTIMOS EL ARCHIVO JSON A UNA DATAFRAME ''

In [54]:
import json
import pandas as pd

with open("vehiculos_registro.json", "r") as f:
    datos = json.load(f)

df_vehiculos = pd.DataFrame(datos)

print(df_vehiculos.head()) 
print("\n\n")
print(df_vehiculos.info())

   PROVINCIA    SERVICIO    CLASE COMBUSTIBLE          MARCA  \
0  SUCUMBIOS    ALQUILER  Trailer      Diesel  INTERNATIONAL   
1     GUAYAS    ALQUILER  Trailer      Diesel  INTERNATIONAL   
2     CARCHI    ALQUILER   Camión      Diesel           HINO   
3   COTOPAXI  PARTICULAR   Camión      Diesel           HINO   
4     EL ORO  PARTICULAR   Camión      Diesel      CHEVROLET   

              MODELO  ASIENTOS  TONELAJE     ESTRATONE ESTRASIENTOS  
0               2006         0      45.0  15 1/4 Y MÁS       0 A 10  
1               2011         0      40.0  15 1/4 Y MÁS       0 A 10  
2  2003 y anteriores         0      12.0   10 1/4 A 15       0 A 10  
3  2003 y anteriores         0       6.5    6 1/4 A 10       0 A 10  
4  2003 y anteriores         0       6.0     3 1/4 A 6       0 A 10  



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125534 entries, 0 to 125533
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -

In [42]:
''PROCEDEMOS A EXPORTAR LOS DATOS DE JUPYTER HACI SQLITE''

In [55]:
# primmero creamos una coneccion a SQLite

registro_sql = "registro_sql.db"
conexion = sqlite3.connect(registro_sql)

In [56]:
# entonces en este paso exportamos nuestra dataframe a sqlite

registro_autos = "registro_autos"
df_vehiculos.to_sql(registro_autos, conexion, if_exists="replace", index=False)

conexion.close()

In [ ]:
'' AHORA TRANSFERIREMOS LOS DATOS DE SQLITE A MYSQL WORKBECH ''

In [20]:
import sqlite3
import pandas as pd

conn_sqlite = sqlite3.connect('registro_sql.db')

In [21]:
registro_autos = pd.read_sql_query("SELECT * FROM registro_autos", conn_sqlite)

In [22]:
from sqlalchemy import create_engine

nombre_usuario = 'root'
contraseña = 'Honkai9029'
nombre_base_datos = 'vehiculos_registro'
host = '127.0.0.1'

engine = create_engine(f'mysql+mysqlconnector://{nombre_usuario}:{contraseña}@{host}/{nombre_base_datos}')

In [ ]:
registro_autos.to_sql('registro_autos', engine, if_exists='replace', index=False)

In [ ]:
''PROCEDEMOS A IMPORTAR LOS DATOS DE SQLITE HACIA JUPYTER''

In [57]:
# se procede a ser la importacion de los datos de sqlite a jupyter

import sqlite3

registro_sql = "registro_sql.db"
conexion = sqlite3.connect(registro_sql)
cursor = conexion.cursor()

cursor.execute("SELECT * FROM registro_autos")
datos = cursor.fetchall()

for i,fila in enumerate(datos):
    if i < 10:
        print(fila)
    else:
        break

conexion.close()

('SUCUMBIOS', 'ALQUILER', 'Trailer', 'Diesel', 'INTERNATIONAL', '2006', 0, 45.0, '15 1/4 Y MÁS', '0 A 10')
('GUAYAS', 'ALQUILER', 'Trailer', 'Diesel', 'INTERNATIONAL', '2011', 0, 40.0, '15 1/4 Y MÁS', '0 A 10')
('CARCHI', 'ALQUILER', 'Camión', 'Diesel', 'HINO', '2003 y anteriores', 0, 12.0, '10 1/4 A 15', '0 A 10')
('COTOPAXI', 'PARTICULAR', 'Camión', 'Diesel', 'HINO', '2003 y anteriores', 0, 6.5, '6 1/4 A 10', '0 A 10')
('EL ORO', 'PARTICULAR', 'Camión', 'Diesel', 'CHEVROLET', '2003 y anteriores', 0, 6.0, '3 1/4 A 6', '0 A 10')
('LOS RIOS', 'PARTICULAR', 'Camión', 'Diesel', 'FORD', '2003 y anteriores', 0, 6.0, '3 1/4 A 6', '0 A 10')
('LOS RIOS', 'PARTICULAR', 'Camión', 'Gasolina', 'FORD', '2003 y anteriores', 0, 5.0, '3 1/4 A 6', '0 A 10')
('MANABI', 'ALQUILER', 'Volqueta', 'Diesel', 'MITSUBISHI', '2003 y anteriores', 0, 5.0, '3 1/4 A 6', '0 A 10')
('CAÑAR', 'PARTICULAR', 'Camión', 'Diesel', 'TOYOTA', '2003 y anteriores', 0, 4.0, '3 1/4 A 6', '0 A 10')
('CHIMBORAZO', 'PARTICULAR', 'Ca

In [16]:
'' CONVERTIMOS LA DATAFRAME  EN JSON ''

In [58]:
df.to_json('vehiculos_json_new.json')

In [ ]:
'' CONVERTIMOS LA DATAFRAME EN CSV ''

In [59]:
df.to_csv('vehiculos_csv_new.csv')